In [ ]:
import matplotlib.path as mplp
import numpy as np
import pandas as pd


df_regions = pd.read_csv("IPCC-WGI-reference-regions-v4_coordinates.csv")

def outline_to_mask(line, x, y):
    """Create mask from outline contour

    Parameters
    ----------
    line: array-like (N, 2)
    x, y: 1-D grid coordinates (input for meshgrid)

    Returns
    -------
    mask : 2-D boolean array (True inside)
    """

    mpath = mplp.Path(line)
    X, Y = np.meshgrid(x, y)
    points = np.array((X.flatten(), Y.flatten())).T
    mask = mpath.contains_points(points).reshape(X.shape)
    return mask

def IPCC_region(data, code):
    """Mask data outside AR6 reference region
    Requires https://github.com/IPCC-WG1/Atlas/blob/devel/reference-regions/IPCC-WGI-reference-regions-v4_coordinates.csv

    Parameters
    ----------
    data: array with shape (ntime,n_latitude,n_longitude)
    code: IPCC AR6 reference region code

    Returns
    -------
    ipcc_masked : array of shape (ntime, n_latitude,n_longitude) with values outside region masked
    """

    vals=df_regions[df_regions.Acronym==code].values[0]
    lons=[]
    lats=[]
    for x in vals:
        if type(x)==type("this"):
            if x.find("|")>0:
                lon,lat=x.split("|")

                lons+=[float(lon)]
                lats+=[float(lat)]
    N = len(lons)
    line = np.zeros((N,2))

    line[:,0] = np.array(lons)
    line[:,1] = np.array(lats)
    x = data.getLatitude()[:] #Edit based on datatype (may gave to change for xarray)
    y = data.getLongitude()[:]
    y = (y+ 180) % 360 - 180
    tmask =~ outline_to_mask(line,y,x)
    tmask.shape
    ipcc_masked = mask_data(data,tmask)
    
    return ipcc_masked
